In [1]:
import pandas as pd
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
import re
import json
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, classification_report

In [2]:
train_df = pd.read_csv("train_ajt_df.csv", index_col=0)
test_df = pd.read_csv("test_ajt_df.csv", index_col=0)

In [3]:
df = pd.read_csv("ajt_dataset.csv")

In [4]:
train_df = pd.concat([train_df, df.loc[train_df.index]['type_mistake'], ], axis=1)
test_df = pd.concat([test_df, df.loc[test_df.index]['type_mistake'], ], axis=1)

In [19]:
test_df['text'].values[3]

'Коррекция звука под особенности помещения давно используется звукорежиссерами для обеспечения настройки звука в студиях и концертных залах, имея целью ослабление влияния особенностей используемого помещения на звук.'

In [5]:
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [6]:
# hf_hub_download(repo_id="bartowski/gemma-2-27b-it-GGUF", filename="gemma-2-27b-it-Q8_0.gguf")
# hf_hub_download(repo_id="bartowski/Qwen2.5-32B-Instruct-GGUF", filename="Qwen2.5-32B-Instruct-Q8_0.gguf")
# hf_hub_download(repo_id="bartowski/gemma-2-9b-it-GGUF", filename="gemma-2-9b-it-Q8_0.gguf")

In [7]:
model_path ='/home/.cache/huggingface/hub/models--bartowski--gemma-2-27b-it-GGUF/snapshots/a8874ad70a9d2ce99d6d41c25920b11bb330aecd/gemma-2-27b-it-Q8_0.gguf'
# model_path = '/home/.cache/huggingface/hub/models--bartowski--Qwen2.5-32B-Instruct-GGUF/snapshots/2116cbb385b8ce3a4d28cf3bf1cd2039a55821a6/Qwen2.5-32B-Instruct-Q8_0.gguf'
# model_path = '/home/.cache/huggingface/hub/models--bartowski--gemma-2-9b-it-GGUF/snapshots/d731033f3dc4018261fd39896e50984d398b4ac5/gemma-2-9b-it-Q8_0.gguf'

In [8]:
llm = Llama(model_path=model_path, n_gpu_layers=-1, n_ctx=2048, verbose=False)

llama_new_context_with_model: n_ctx_per_seq (2048) < n_ctx_train (8192) -- the full capacity of the model will not be utilized


In [9]:
def generate_llm_answer(system_prompt, input_txt, model_like='other'):
    if model_like == 'gemma':
        messages=[
        {"role": "user", "content": system_prompt + input_txt},
    ]
    else:
        messages=[
        {
            "role": "system",
            "content": system_prompt,
        },
        {   "role": "user", 
            "content": input_txt
        },
    ]

    output = llm.create_chat_completion(
        messages,
        temperature=0.1,
        top_p=0.9,
        top_k=40,
        repeat_penalty=1.05,
        )
    
    gen_answ = output['choices'][0]['message']['content']
    # print("Query:", input_txt)
    # print("Generated answer:", gen_answ)
    # print('\n\n')
    return gen_answ

In [10]:
def extract_answer_from_json(answer):
    json_pattern = re.search(r'```json\n(.*?)\n```', answer, re.DOTALL)
    extracted_json_string = json_pattern.group(1) if json_pattern else None
    res = json.loads(extracted_json_string)
    return res

# ZeRo-shot

## Binary clf and type mistake w no mistakes

In [10]:
system_prompt = """ --Цель--  
Ты - квалифицированный лингвист. Читай очень внимательно. Оцени приемлемость текста, написанного студентом технического профиля. 
Определи, содержит ли текст ошибку (1 — есть, 0 — нет). Если ошибка есть, классифицируй её как одну из следующих категорий:

- Речевая: неправильный выбор слов, канцеляризмы, некорректные выражения.
- Логическая: нарушение логики изложения, противоречия.
- Стилистическая:несоответствие стилю научного/технического текста.
- Грамматическая: ошибки в склонении, спряжении, согласовании слов.
- Пунктуационная: пропущенные или избыточные знаки препинания.
- Лексическая: неправильный выбор слова.

--Формат ответа-- 

Оъяснение ответа не нужно!

- Выведи ключевые слова в формате JSON.
- JSON должен содержать два ключа:
  - "is_mistake_pred"
  - "error_type_pred"

Если ошибка есть:

```json
{
  "is_mistake_pred": 1,
  "error_type_pred": "тип ошибки"
}
```

Если ошибки нет:

```json
{
  "is_mistake_pred": 0,
  "error_type_pred": "нет ошибки"
}
```

Текст: 
"""

In [11]:
preds = []
for i, v in tqdm(enumerate(test_df.text.values), total=len(test_df)):
    out = generate_llm_answer(system_prompt, v, model_like='gemma')
    try:
        struct_out = extract_answer_from_json(out)
        preds.append(struct_out)
    except Exception as e:
        print(f"Error extractuin json from index {i}: {e}")
        bad_output = {"is_mistake_pred": 1, "error_type_pred": "не определена"}
        preds.append(bad_output)

  0%|          | 0/197 [00:00<?, ?it/s]

In [12]:
pred_df = pd.DataFrame(preds)
pred_df['error_type_pred'] = pred_df['error_type_pred'].apply(lambda x: x.lower().strip())
pred_df.head()

,is_mistake_pred,error_type_pred
0,0,нет ошибки
1,1,стилистическая
2,1,стилистическая
3,1,стилистическая
4,1,стилистическая


In [13]:
print(f"Accuracy/Binary: {accuracy_score(test_df['is_mistake'], pred_df['is_mistake_pred'])}")
print(f"F1/Binary: {f1_score(test_df['is_mistake'], pred_df['is_mistake_pred'])}")
print(f"MCC/Binray: {matthews_corrcoef(test_df['is_mistake'], pred_df['is_mistake_pred'])}")

Accuracy/Binary: 0.649746192893401
F1/Binary: 0.6698564593301436
MCC/Binray: 0.3118730722792103


In [14]:
label_mapper = {"нет ошибки": 0, "речевая": 1, "стилистическая": 2, "пунктуационная": 3, "грамматическая": 4, "лексическая": 5, "логическая": 6}
reverse_label_mapper = {0: "нет ошибки", 1: "речевая", 2: "стилистическая", 3: "пунктуационная", 4: "грамматическая", 5: "лексическая", 6: "логическая"}

In [15]:
pred_df['error_type_pred'] = pred_df['error_type_pred'].map(label_mapper)
test_df['type_mistake'] = test_df['type_mistake'].map(label_mapper)

In [16]:
# w no mistakes
print(f"Accuracy/Multi: {accuracy_score(test_df['type_mistake'], pred_df['error_type_pred'])}")
print(f"F1/Multi: {f1_score(test_df['type_mistake'], pred_df['error_type_pred'], average='macro')}")
print(f"MCC/Multi: {matthews_corrcoef(test_df['type_mistake'], pred_df['error_type_pred'])}")

Accuracy/Multi: 0.36548223350253806
F1/Multi: 0.15227370227370227
MCC/Multi: 0.13590894836915793


## Only binary

In [17]:
system_prompt = """ --Цель--  
Ты - квалифицированный лингвист. Читай очень внимательно. Оцени приемлемость текста, написанного студентом технического профиля. 
Определи, содержит ли текст ошибку (1 — есть, 0 — нет). Ошибки могут быть следующего вида:

- Речевая: неправильный выбор слов, канцеляризмы, некорректные выражения.
- Логическая: нарушение логики изложения, противоречия.
- Стилистическая:несоответствие стилю научного/технического текста.
- Грамматическая: ошибки в склонении, спряжении, согласовании слов.
- Пунктуационная: пропущенные или избыточные знаки препинания.
- Лексическая: неправильный выбор слова.

--Формат ответа-- 

Оъяснение ответа не нужно!

- Выведи ключевые слова в формате JSON.
- JSON должен содержать ключ:
  - "is_mistake_pred"

Пример:

```json
{
  "is_mistake_pred": "1 или 0"
}
```

Текст: 
"""

In [18]:
preds = []
for i, v in tqdm(enumerate(test_df.text.values), total=len(test_df)):
    out = generate_llm_answer(system_prompt, v, model_like='gemma')
    try:
        struct_out = extract_answer_from_json(out)
        preds.append(struct_out)
    except Exception as e:
        print(f"Error extractuin json from index {i}: {e}")
        bad_output = {"is_mistake_pred": 1}
        preds.append(bad_output)

  0%|          | 0/197 [00:00<?, ?it/s]

In [19]:
pred_df = pd.DataFrame(preds)
pred_df['is_mistake_pred'] = pred_df['is_mistake_pred'].apply(lambda x: float(x))
pred_df.head()

,is_mistake_pred
0,0.0
1,1.0
2,0.0
3,1.0
4,1.0


In [20]:
print(f"Accuracy/Binary: {accuracy_score(test_df['is_mistake'], pred_df['is_mistake_pred'])}")
print(f"F1/Binary: {f1_score(test_df['is_mistake'], pred_df['is_mistake_pred'])}")
print(f"MCC/Binray: {matthews_corrcoef(test_df['is_mistake'], pred_df['is_mistake_pred'])}")

Accuracy/Binary: 0.6751269035532995
F1/Binary: 0.6483516483516484
MCC/Binray: 0.3477261111996992


## Multi-class w/o no mistakes

In [10]:
system_prompt = """ --Цель--  
Ты - квалифицированный лингвист. Читай очень внимательно. Тебе необходимо классифицировать ошибку в тексте на одну из следующих категорий:

- Речевая: неправильный выбор слов, канцеляризмы, некорректные выражения.
- Логическая: нарушение логики изложения, противоречия.
- Стилистическая:несоответствие стилю научного/технического текста.
- Грамматическая: ошибки в склонении, спряжении, согласовании слов.
- Пунктуационная: пропущенные или избыточные знаки препинания.
- Лексическая: неправильный выбор слова.

--Формат ответа-- 

Оъяснение ответа не нужно!

- Выведи ключевые слова в формате JSON.
- JSON должен содержать ключ:
  - "error_type_pred"

Пример:

```json
{
  "error_type_pred": "тип ошибки"
}
```

Текст: 
"""

In [11]:
train_df = train_df[train_df['type_mistake'] != "нет ошибки"]
test_df = test_df[test_df['type_mistake'] != "нет ошибки"]

In [12]:
preds = []
for i, v in tqdm(enumerate(test_df.text.values), total=len(test_df)):
    out = generate_llm_answer(system_prompt, v, model_like='gemma')
    try:
        struct_out = extract_answer_from_json(out)
        preds.append(struct_out)
    except Exception as e:
        print(f"Error extractuin json from index {i}: {e}")
        bad_output = {"error_type_pred": "не определена"}
        preds.append(bad_output)

  0%|          | 0/94 [00:00<?, ?it/s]

In [13]:
pred_df = pd.DataFrame(preds)
pred_df['error_type_pred'] = pred_df['error_type_pred'].apply(lambda x: x.lower().strip())
pred_df.head()

,error_type_pred
0,стилистическая
1,стилистическая
2,стилистическая
3,стилистическая
4,лексическая


In [14]:
pred_df['error_type_pred'] = pred_df['error_type_pred'].apply(lambda x: "грамматическая" if x == "орфографическая" else x)

In [15]:
label_mapper = {"речевая": 0, "стилистическая": 1, "пунктуационная": 2, "грамматическая": 3, "лексическая": 4, "логическая": 5}
reverse_label_mapper = {0: "речевая", 1: "стилистическая", 2: "пунктуационная", 3: "грамматическая", 4: "лексическая", 5: "логическая"}

In [16]:
pred_df['error_type_pred'] = pred_df['error_type_pred'].map(label_mapper)
test_df['type_mistake'] = test_df['type_mistake'].map(label_mapper)

In [17]:
print(f"Accuracy/Multi: {accuracy_score(test_df['type_mistake'], pred_df['error_type_pred'])}")
print(f"F1/Multi: {f1_score(test_df['type_mistake'], pred_df['error_type_pred'], average='macro')}")
print(f"MCC/Multi: {matthews_corrcoef(test_df['type_mistake'], pred_df['error_type_pred'])}")

Accuracy/Multi: 0.23404255319148937
F1/Multi: 0.20826397854906628
MCC/Multi: 0.05803390503496998


# Few-shot

## Binary clf and type mistake w no mistakes

In [31]:
system_prompt = """ --Цель--  
Ты - квалифицированный лингвист. Читай очень внимательно. Оцени приемлемость текста, написанного студентом технического профиля. 
Определи, содержит ли текст ошибку (1 — есть, 0 — нет). Если ошибка есть, классифицируй её как одну из следующих категорий:

- Речевая: неправильное построение фразы или нарушение логики высказывания.

Пример 1:
Текст: Международная организация, выпускающая стандарты, привнесла значительный вклад в стандартизацию управления сетевым оборудованием.
Исправленный текст: Международная организация, выпускающая стандарты, внесла значительный вклад в стандартизацию управления сетевым оборудованием.
Объяснение: Употребление слова в несвойственном контексте.  

Пример 2:
Текст: В работе проведен анализ и исследование методов испытаний пироустройств на исправность.
Исправленный текст: В работе проведен анализ и исследование методов испытаний пироустройств.
Объяснение: Неверная речевая конструкция.

Пример 3:
Текст: Рассмотренные в данной работе гиперграфы представляют из себя расширение традиционной теории графов в случае, когда ребро графа содержит более двух вершин.
Исправленный текст: Рассмотренные в данной работе гиперграфы представляют из себя расширение традиционной теории графов в случае, когда его ребро содержит более двух вершин.
Объяснение: Тавтология.

- Логическая: нарушение логики изложения, противоречия.

Пример 1: 
Текст: Помимо очевидных плюсов, есть так же побочные минусы.
Исправленный текст: Помимо очевидных достоинств технологического развития, имеются также недостатки.
Объяснение: Не указан процесс, о котором говорится в предложении.

Пример 2:
Текст: Это достигается за счет добавления избыточных битов при через канал связи.
Исправленный текст: Это достигается за счет добавления избыточных битов при передаче через канал связи.
Объяснение: Пропущено слово.

Пример 3:
Текст: Таким образом, работа будет посвящена изучению LDPC-LDGM каскадных кодов, сравнение данного каскада с LDPC-SPARC-кодом.
Исправленный текст: Таким образом, работа посвящена изучению LDPC-LDGM каскадных кодов, сравнению данного каскада с LDPC-SPARC-кодом.
Объяснение: Используется будущее время "работа будет посвящена".

- Стилистическая: несоответствие стилю научного/технического текста.

Пример 1: 
Текст: Как видно на схеме, я использую только один канал усилителя TL072.
Исправленный текст: Согласно схеме, задействован только один канал усилителя TL072.
Объяснение: Нехарактерный для научного стиля оборот.

Пример 2:
Текст: Этот процесс имеет два основных преимущества: ...
Исправленный текст: У этого процесса можно выделить два основных преимущества: …
Объяснение: Применено слово "имеет".

Пример 3:
Текст: Телефонные линии голосовой связи, которые были популярны около 15 лет назад, частично заменены сегодня Интернет-телефонией.
Исправленный текст: Телефонные линии голосовой связи, которые были популярны около 15 лет назад, сегодня постепенно заменяются интернет-телефонией.
Объяснение: Неудачное использование оборота "частично заменены" в контексте с "сегодня".

- Грамматическая: ошибки в склонении, спряжении, согласовании слов.

Пример 1:
Текст: Защита служебной тайны должна строится на основе комбинированного подхода, …
Исправленный текст: Защита служебной тайны должна строиться на основе комбинированного подхода, …
Объяснение: "Строится" вместо "строиться".

Пример 2:
Текст: Для них выделены два персональных компьютера, осуществляющие вывод изображения на мониторы.
Исправленный текст: Для них выделены два персональных компьютера, осуществляющих вывод изображения на мониторы.
Объяснение: Неверно выбрана падежная форма причастия.

Пример 3:
Текст: Существует несколько исследования, схожих с теми, которые выполняются в этой работе.
Исправленный текст: Существует несколько исследований, схожих с теми, которые выполняются в этой работе.
Объяснение: Ошибка в согласовании членов.

- Пунктуационная: пропущенные или избыточные знаки препинания.

Пример 1:
Текст: Приложение-контроллер системы "подписывается" на определенные темы (topics), …
Исправленный текст: Приложение-контроллер системы «подписывается» на определенные темы (topics), …
Объяснение: Неправильные кавычки.

Пример 2:
Текст: Вследствие чего для тестирования робота как системы необходимо применять комбинацию стандартов предъявляемых к его составным, частям, …
Исправленный текст: Вследствие чего для тестирования робота как системы необходимо применять комбинацию стандартов, предъявляемых к его составным частям, …
Объяснение: Не хватает запятой в причастном обороте.

Пример 3
Текст: Отслеживание таких событий вручную требует внимания членов съемочной группы, или даже специально выделенных под эту задачу сотрудников.
Исправленный текст: Отслеживание таких событий вручную требует внимания членов съемочной группы или даже специально выделенных под эту задачу сотрудников.
Объяснение: Лишняя запятая между однородными членами предложения, соединёнными союзом "или".

- Лексическая: неправильный выбор слова.

Пример 1:
Текст: С развитием цифровых технологий и расширением интернета стало намного проще обмениваться данными.
Исправленный текст: С развитием цифровых технологий и распространением интернета стало проще обмениваться данными.
Объяснение: Неверный выбор выражения "расширение интернета".

Пример 2:
Текст: Необходимо внести суть графика в его названии (заголовке).
Исправленный текст: Необходимо отразить суть графика в его названии (заголовке).
Объяснение: Использование слова "внести" в несвойственном значении.

Пример 3:
Текст: Используя Python и среду разработки PyCharm, были созданы модели для определения необходимого количества базовых станций и их радиуса действия для обеспечения надёжного соединения.
Исправленный текст: Используя Python и среду PyCharm, были разработаны модели для определения необходимого количества базовых станций и их радиуса действия для обеспечения надёжного соединения.
Объяснение: Программы разрабатываются а не создаются.

--Примеры без ошибок--

Пример 1: 
Текст: Эта проблема была решена использованием нейронных сетей глубокого обучения.

Пример 2: 
Текст: Такие автономные системы соединены с инфраструктурными IXP и имеют глобальный или континентальный географический охват.

Пример 3: 
Текст: В этих системах в основном отсутствуют встроенные средства защиты от утечек информации.

--Формат ответа-- 

Оъяснение ответа не нужно!

- Выведи ключевые слова в формате JSON.
- JSON должен содержать два ключа:
  - "is_mistake_pred"
  - "error_type_pred"

Если ошибка есть:

```json
{
  "is_mistake_pred": 1,
  "error_type_pred": "тип ошибки"
}
```

Если ошибки нет:

```json
{
  "is_mistake_pred": 0,
  "error_type_pred": "нет ошибки"
}
```

Текст: 
"""

In [32]:
preds = []
for i, v in tqdm(enumerate(test_df.text.values), total=len(test_df)):
    out = generate_llm_answer(system_prompt, v, model_like='gemma')
    try:
        struct_out = extract_answer_from_json(out)
        preds.append(struct_out)
    except Exception as e:
        print(f"Error extractuin json from index {i}: {e}")
        bad_output = {"is_mistake_pred": 1, "error_type_pred": "не определена"}
        preds.append(bad_output)

  0%|          | 0/197 [00:00<?, ?it/s]

In [12]:
pred_df = pd.DataFrame(preds)
pred_df['error_type_pred'] = pred_df['error_type_pred'].apply(lambda x: x.lower().strip())
pred_df.head()

,is_mistake_pred,error_type_pred
0,0,нет ошибки
1,1,пунктуационная
2,1,стилистическая
3,1,речевая
4,1,лексическая


In [14]:
pred_df['error_type_pred'] = pred_df['error_type_pred'].apply(lambda x: "грамматическая" if x == "орфографическая" else x)

In [15]:
print(f"Accuracy/Binary: {accuracy_score(test_df['is_mistake'], pred_df['is_mistake_pred'])}")
print(f"F1/Binary: {f1_score(test_df['is_mistake'], pred_df['is_mistake_pred'])}")
print(f"MCC/Binray: {matthews_corrcoef(test_df['is_mistake'], pred_df['is_mistake_pred'])}")

Accuracy/Binary: 0.6700507614213198
F1/Binary: 0.6666666666666666
MCC/Binray: 0.34172782166446486


In [16]:
label_mapper = {"нет ошибки": 0, "речевая": 1, "стилистическая": 2, "пунктуационная": 3, "грамматическая": 4, "лексическая": 5, "логическая": 6}
reverse_label_mapper = {0: "нет ошибки", 1: "речевая", 2: "стилистическая", 3: "пунктуационная", 4: "грамматическая", 5: "лексическая", 6: "логическая"}

In [17]:
pred_df['error_type_pred'] = pred_df['error_type_pred'].map(label_mapper)
test_df['type_mistake'] = test_df['type_mistake'].map(label_mapper)

In [18]:
# w no mistakes
print(f"Accuracy/Multi: {accuracy_score(test_df['type_mistake'], pred_df['error_type_pred'])}")
print(f"F1/Multi: {f1_score(test_df['type_mistake'], pred_df['error_type_pred'], average='macro')}")
print(f"MCC/Multi: {matthews_corrcoef(test_df['type_mistake'], pred_df['error_type_pred'])}")

Accuracy/Multi: 0.41624365482233505
F1/Multi: 0.21599074431989004
MCC/Multi: 0.17243656541738137


## Only binary

In [14]:
system_prompt = """ --Цель--  
Ты - квалифицированный лингвист. Читай очень внимательно. Оцени приемлемость текста, написанного студентом технического профиля. 
Определи, содержит ли текст ошибку (1 — есть, 0 — нет). Ошибки могут быть следующего вида:

- Речевая: неправильное построение фразы или нарушение логики высказывания.

Пример:
Текст: В работе проведен анализ и исследование методов испытаний пироустройств на исправность.
Исправленный текст: В работе проведен анализ и исследование методов испытаний пироустройств.
Объяснение: Неверная речевая конструкция.

- Логическая: нарушение логики изложения, противоречия.

Пример:
Текст: Это достигается за счет добавления избыточных битов при через канал связи.
Исправленный текст: Это достигается за счет добавления избыточных битов при передаче через канал связи.
Объяснение: Пропущено слово.

- Стилистическая: несоответствие стилю научного/технического текста.

Пример:
Текст: Этот процесс имеет два основных преимущества: ...
Исправленный текст: У этого процесса можно выделить два основных преимущества: …
Объяснение: Применено слово "имеет".

- Грамматическая: ошибки в склонении, спряжении, согласовании слов.

Пример:
Текст: Для них выделены два персональных компьютера, осуществляющие вывод изображения на мониторы.
Исправленный текст: Для них выделены два персональных компьютера, осуществляющих вывод изображения на мониторы.
Объяснение: Неверно выбрана падежная форма причастия.

- Пунктуационная: пропущенные или избыточные знаки препинания.

Пример:
Текст: Вследствие чего для тестирования робота как системы необходимо применять комбинацию стандартов предъявляемых к его составным, частям, …
Исправленный текст: Вследствие чего для тестирования робота как системы необходимо применять комбинацию стандартов, предъявляемых к его составным частям, …
Объяснение: Не хватает запятой в причастном обороте.

- Лексическая: неправильный выбор слова.

Пример:
Текст: Необходимо внести суть графика в его названии (заголовке).
Исправленный текст: Необходимо отразить суть графика в его названии (заголовке).
Объяснение: Использование слова "внести" в несвойственном значении.

--Примеры без ошибок--

Пример 1: 
Текст: Эта проблема была решена использованием нейронных сетей глубокого обучения.

Пример 2: 
Текст: Такие автономные системы соединены с инфраструктурными IXP и имеют глобальный или континентальный географический охват.

Пример 3: 
Текст: В этих системах в основном отсутствуют встроенные средства защиты от утечек информации.

--Формат ответа-- 

Оъяснение ответа не нужно!

- Выведи ключевые слова в формате JSON.
- JSON должен содержать ключ:
  - "is_mistake_pred"

Пример:

```json
{
  "is_mistake_pred": "1 или 0"
}
```

Текст: 
"""

In [15]:
preds = []
for i, v in tqdm(enumerate(test_df.text.values), total=len(test_df)):
    out = generate_llm_answer(system_prompt, v, model_like='gemma')
    try:
        struct_out = extract_answer_from_json(out)
        preds.append(struct_out)
    except Exception as e:
        print(f"Error extractuin json from index {i}: {e}")
        bad_output = {"is_mistake_pred": 1}
        preds.append(bad_output)

  0%|          | 0/197 [00:00<?, ?it/s]

In [15]:
pred_df = pd.DataFrame(preds)
pred_df['is_mistake_pred'] = pred_df['is_mistake_pred'].apply(lambda x: float(x))
pred_df.head()

,is_mistake_pred
0,0.0
1,1.0
2,1.0
3,1.0
4,1.0


In [16]:
print(f"Accuracy/Binary: {accuracy_score(test_df['is_mistake'], pred_df['is_mistake_pred'])}")
print(f"F1/Binary: {f1_score(test_df['is_mistake'], pred_df['is_mistake_pred'])}")
print(f"MCC/Binray: {matthews_corrcoef(test_df['is_mistake'], pred_df['is_mistake_pred'])}")

Accuracy/Binary: 0.6446700507614214
F1/Binary: 0.6022727272727273
MCC/Binray: 0.2860231901137858


## Multi-class w/o no mistakes

In [11]:
system_prompt = """ --Цель--  
Ты - квалифицированный лингвист. Читай очень внимательно. Классифицируй ошибку в тексте как одну из следующих категорий:

- Речевая: неправильное построение фразы или нарушение логики высказывания.

Пример 1:
Текст: Международная организация, выпускающая стандарты, привнесла значительный вклад в стандартизацию управления сетевым оборудованием.
Исправленный текст: Международная организация, выпускающая стандарты, внесла значительный вклад в стандартизацию управления сетевым оборудованием.
Объяснение: Употребление слова в несвойственном контексте.  

Пример 2:
Текст: В работе проведен анализ и исследование методов испытаний пироустройств на исправность.
Исправленный текст: В работе проведен анализ и исследование методов испытаний пироустройств.
Объяснение: Неверная речевая конструкция.

Пример 3:
Текст: Рассмотренные в данной работе гиперграфы представляют из себя расширение традиционной теории графов в случае, когда ребро графа содержит более двух вершин.
Исправленный текст: Рассмотренные в данной работе гиперграфы представляют из себя расширение традиционной теории графов в случае, когда его ребро содержит более двух вершин.
Объяснение: Тавтология.

- Логическая: нарушение логики изложения, противоречия.

Пример 1: 
Текст: Помимо очевидных плюсов, есть так же побочные минусы.
Исправленный текст: Помимо очевидных достоинств технологического развития, имеются также недостатки.
Объяснение: Не указан процесс, о котором говорится в предложении.

Пример 2:
Текст: Это достигается за счет добавления избыточных битов при через канал связи.
Исправленный текст: Это достигается за счет добавления избыточных битов при передаче через канал связи.
Объяснение: Пропущено слово.

Пример 3:
Текст: Таким образом, работа будет посвящена изучению LDPC-LDGM каскадных кодов, сравнение данного каскада с LDPC-SPARC-кодом.
Исправленный текст: Таким образом, работа посвящена изучению LDPC-LDGM каскадных кодов, сравнению данного каскада с LDPC-SPARC-кодом.
Объяснение: Используется будущее время "работа будет посвящена".

- Стилистическая: несоответствие стилю научного/технического текста.

Пример 1: 
Текст: Как видно на схеме, я использую только один канал усилителя TL072.
Исправленный текст: Согласно схеме, задействован только один канал усилителя TL072.
Объяснение: Нехарактерный для научного стиля оборот.

Пример 2:
Текст: Этот процесс имеет два основных преимущества: ...
Исправленный текст: У этого процесса можно выделить два основных преимущества: …
Объяснение: Применено слово "имеет".

Пример 3:
Текст: Телефонные линии голосовой связи, которые были популярны около 15 лет назад, частично заменены сегодня Интернет-телефонией.
Исправленный текст: Телефонные линии голосовой связи, которые были популярны около 15 лет назад, сегодня постепенно заменяются интернет-телефонией.
Объяснение: Неудачное использование оборота "частично заменены" в контексте с "сегодня".

- Грамматическая: ошибки в склонении, спряжении, согласовании слов.

Пример 1:
Текст: Защита служебной тайны должна строится на основе комбинированного подхода, …
Исправленный текст: Защита служебной тайны должна строиться на основе комбинированного подхода, …
Объяснение: "Строится" вместо "строиться".

Пример 2:
Текст: Для них выделены два персональных компьютера, осуществляющие вывод изображения на мониторы.
Исправленный текст: Для них выделены два персональных компьютера, осуществляющих вывод изображения на мониторы.
Объяснение: Неверно выбрана падежная форма причастия.

Пример 3:
Текст: Существует несколько исследования, схожих с теми, которые выполняются в этой работе.
Исправленный текст: Существует несколько исследований, схожих с теми, которые выполняются в этой работе.
Объяснение: Ошибка в согласовании членов.

- Пунктуационная: пропущенные или избыточные знаки препинания.

Пример 1:
Текст: Приложение-контроллер системы "подписывается" на определенные темы (topics), …
Исправленный текст: Приложение-контроллер системы «подписывается» на определенные темы (topics), …
Объяснение: Неправильные кавычки.

Пример 2:
Текст: Вследствие чего для тестирования робота как системы необходимо применять комбинацию стандартов предъявляемых к его составным, частям, …
Исправленный текст: Вследствие чего для тестирования робота как системы необходимо применять комбинацию стандартов, предъявляемых к его составным частям, …
Объяснение: Не хватает запятой в причастном обороте.

Пример 3
Текст: Отслеживание таких событий вручную требует внимания членов съемочной группы, или даже специально выделенных под эту задачу сотрудников.
Исправленный текст: Отслеживание таких событий вручную требует внимания членов съемочной группы или даже специально выделенных под эту задачу сотрудников.
Объяснение: Лишняя запятая между однородными членами предложения, соединёнными союзом "или".

- Лексическая: неправильный выбор слова.

Пример 1:
Текст: С развитием цифровых технологий и расширением интернета стало намного проще обмениваться данными.
Исправленный текст: С развитием цифровых технологий и распространением интернета стало проще обмениваться данными.
Объяснение: Неверный выбор выражения "расширение интернета".

Пример 2:
Текст: Необходимо внести суть графика в его названии (заголовке).
Исправленный текст: Необходимо отразить суть графика в его названии (заголовке).
Объяснение: Использование слова "внести" в несвойственном значении.

Пример 3:
Текст: Используя Python и среду разработки PyCharm, были созданы модели для определения необходимого количества базовых станций и их радиуса действия для обеспечения надёжного соединения.
Исправленный текст: Используя Python и среду PyCharm, были разработаны модели для определения необходимого количества базовых станций и их радиуса действия для обеспечения надёжного соединения.
Объяснение: Программы разрабатываются а не создаются.

--Формат ответа-- 

Оъяснение ответа не нужно!

- Выведи ключевые слова в формате JSON.
- JSON должен содержать ключ:
  - "error_type_pred"

Пример: 

```json
{
  "error_type_pred": "тип ошибки"
}
```

Текст: 
"""

In [12]:
train_df = train_df[train_df['type_mistake'] != "нет ошибки"]
test_df = test_df[test_df['type_mistake'] != "нет ошибки"]

In [13]:
preds = []
for i, v in tqdm(enumerate(test_df.text.values), total=len(test_df)):
    out = generate_llm_answer(system_prompt, v, model_like='gemma')
    try:
        struct_out = extract_answer_from_json(out)
        preds.append(struct_out)
    except Exception as e:
        print(f"Error extractuin json from index {i}: {e}")
        bad_output = {"error_type_pred": "не определена"}
        preds.append(bad_output)

  0%|          | 0/94 [00:00<?, ?it/s]

In [14]:
pred_df = pd.DataFrame(preds)
pred_df['error_type_pred'] = pred_df['error_type_pred'].apply(lambda x: x.lower().strip())
pred_df.head()

,error_type_pred
0,стилистическая
1,стилистическая
2,речевая
3,лексическая
4,лексическая


In [15]:
pred_df['error_type_pred'] = pred_df['error_type_pred'].apply(lambda x: "грамматическая" if x == "орфографическая" else x)

In [16]:
label_mapper = {"речевая": 0, "стилистическая": 1, "пунктуационная": 2, "грамматическая": 3, "лексическая": 4, "логическая": 5}
reverse_label_mapper = {0: "речевая", 1: "стилистическая", 2: "пунктуационная", 3: "грамматическая", 4: "лексическая", 5: "логическая"}

In [17]:
pred_df['error_type_pred'] = pred_df['error_type_pred'].map(label_mapper)
test_df['type_mistake'] = test_df['type_mistake'].map(label_mapper)

In [18]:
print(f"Accuracy/Multi: {accuracy_score(test_df['type_mistake'], pred_df['error_type_pred'])}")
print(f"F1/Multi: {f1_score(test_df['type_mistake'], pred_df['error_type_pred'], average='macro')}")
print(f"MCC/Multi: {matthews_corrcoef(test_df['type_mistake'], pred_df['error_type_pred'])}")

Accuracy/Multi: 0.23404255319148937
F1/Multi: 0.18771919692972325
MCC/Multi: 0.06807212487652928
